In [1]:
import re
import numpy as np
from os import path, listdir
import pandas as pd

In [2]:
def load_data(data_path, data_file):
    # load data from csv
    df = pd.read_csv(path.join(data_path, data_file))
    return df

def adjust_time(df):
    # compute max fruit harvest
    max_fruit_gr = 0.328*1e-6 # kg [DW] m-2 s-1
    delta_t = 300
    max_fruit_gr *= delta_t
    max_fruit_gr

    df['Time'] = df['Time'].map(lambda x: str(x)[:-6])
    df["Fruit harvest norm"] = df["Fruit harvest"] / max_fruit_gr
    df['Date'] = pd.to_datetime(df['Time'])
    df['TimeOfDay'] = df['Date'].dt.time
    return df

def compute_profit_eps(df):
    # compute profit per episode
    N = (df[df['episode'] == 0]).shape[0]
    profits_per_episode = df[['Profits', 'episode']].groupby('episode').sum().reset_index()
    return profits_per_episode

def extract_data(data_path, last):
    runnames = listdir(data_path)
    runnames = [runname for runname in runnames if '60' in runname]
    if last:
        runnames = [runname for runname in runnames if 'last' in runname]
    else:
        runnames = [runname for runname in runnames if 'best' in runname]

    runnames = sorted(runnames, key=lambda x: int(re.findall(r'\d+', x)[0]))
    print(runnames)
    # print(runnames)
    # sort run on the first integer in their name, use regex
    # print(runnames)
    
    dfs = [load_data(data_path, runname) for runname in runnames]
    dfs = [adjust_time(df) for df in dfs]
    return dfs, runnames

def compute_stats(dfs):
    returns = [df['Final return'].unique() for df in dfs]
    returns = [np.sort(ret) for ret in returns]
    means = np.mean(returns, axis=1)
    args = np.argsort(means)[::-1]
    quartile1, medians, quartile3 = np.percentile(returns, [25, 50, 75], axis=1)
    return quartile1[:], medians[:], quartile3[:], args[:], returns[:]
    # returns = [returns[i] for i in args]



In [3]:

def aggregate_data(df: pd.DataFrame, column: str) -> pd.DataFrame:
    """
    Function that computes statistics for violations and profits per episode.
    This function takes in a DataFrame and the name of the column to be used for violations.

    Args:
    - df: the DataFrame
    - column: the name of the column to be used for violations

    Returns:
    - episode: the episode number
    - Profits: the total profits for the episode
    - CO2 Violation Time (%): the percentage of time with CO2 violations
    - CO2 Violation (ppm): the average magnitude of CO2 violations
    """
    # print(df)
    N = (df[df['episode'] == 0]).shape[0]
    profits_per_episode = df[['Profits', 'episode']].groupby('episode').sum().reset_index()
    # CO2 violation time per episode, considering each row as 5 minutes
    co2_violation_time_updated = df[df[column] > 0].groupby('episode').size()/N*100 # % of time with violation
    # Average magnitude of CO2 violations per episode, for positive violations only
    # avg_co2_violation_magnitude_updated = df[[column, 'episode']].groupby('episode')[column].sum()
    avg_co2_violation_magnitude_updated = df[df[column] > 0].groupby('episode')[column].mean()
    # Combine the updated results into a summary DataFrame
    summary_df_updated = pd.DataFrame({
        f'Time within boundary (%)': co2_violation_time_updated,
        f'{column} (abs)': avg_co2_violation_magnitude_updated,
    }).reset_index()


    # Create a DataFrame of all unique episodes to ensure all are represented
    all_episodes_df = pd.DataFrame(df['episode'].unique(), columns=['episode'])

    # Merge the summary of violations with the complete list of episodes
    # This ensures episodes with no violations are included, filling missing values appropriately
    full_summary_df = pd.merge(all_episodes_df, summary_df_updated, on='episode', how='left').fillna(0)
    # print(full_summary_df['coefficients'])
    full_summary_df = pd.merge(profits_per_episode, full_summary_df, on='episode', how='left').fillna(0)
    full_summary_df['Time within boundary (%)'] = 100- full_summary_df['Time within boundary (%)']
    return full_summary_df

def calculate_twb(dataframes, labels):
    twb_df = pd.DataFrame()
    twb_df_ci = pd.DataFrame()
    N = dataframes[0]['episode'].unique().shape[0]
    for j, df in enumerate(dataframes):
        vars = ['CO2 violation', 'Temperature violation', 'Humidity violation']
        violations = [aggregate_data(df, var) for var in vars]

        twb = np.array([violations[i]['Time within boundary (%)'].mean() for i in range(len(vars))])
        df_twb = pd.DataFrame({labels[j]: twb,}, index=vars)
        twb_df = pd.concat([twb_df, df_twb], axis=1)
        
        cis = [ci(violations[i]['Time within boundary (%)'].std(), N) for i in range(len(vars))]
        df_twb_ci = pd.DataFrame({labels[j]: cis,}, index=vars)
        twb_df_ci = pd.concat([twb_df_ci, df_twb_ci], axis=1)
    return twb_df.T, twb_df_ci.T

def ci(std, n, z=2.576):
    return z*std/np.sqrt(n)

In [4]:
def compute_profits_twb(path_name):
    dfs, runnames = extract_data(path_name, last=True)
    # additive_dfs, additive_runnames = extract_data(additive_path, last=True)

    profits_per_episode = [compute_profit_eps(df) for df in dfs]
    twb_df, twb_df_ci = calculate_twb(dfs, runnames)
    return profits_per_episode, twb_df, twb_df_ci, dfs
    # additive_profits_per_episode = [compute_profit_eps(df) for df in additive_dfs]
    # additive_twb_df, additive_twb_df_ci = calculate_twb(additive_dfs, additive_runnames)


def compute_stats(dfs):
    returns = [df['Final return'].unique() for df in dfs]
    returns = [np.sort(ret) for ret in returns]
    means = np.mean(returns, axis=1)
    args = np.argsort(means)[::-1]
    quartile1, medians, quartile3 = np.percentile(returns, [25, 50, 75], axis=1)
    return quartile1[:], medians[:], quartile3[:], args[:], returns[:]
    # returns = [returns[i] for i in args]


# Extract data for test set

In [5]:
multiplicative_path =  '../data/benchmark/test/multiplicative-0.99'
additive_path =  '../data/benchmark/test/additive-0.99'
rule_based_path =  '../data/benchmark/test/rule-based'

test_multi_profits_per_episode, test_multi_twb_df, test_multi_twb_df_ci, test_multi_dfs = compute_profits_twb(multiplicative_path)
test_multi_returns = compute_stats(test_multi_dfs)[-1]

test_additive_profits_per_episode, test_addtive_twb_df, test_addtive_twb_df_ci, test_additive_dfs = compute_profits_twb(additive_path)
test_additive_returns = compute_stats(test_additive_dfs)[-1]

test_rule_based_profits_per_episode, test_rule_based_twb_df, test_addtive_twb_df_ci, rule_based_multi_dfs = compute_profits_twb(rule_based_path)
# test_rule_based_returns = compute_stats(rule_based_multi_dfs)[-1]

['elegant-carrier-2-60-last.csv', 'rogue-podracer-3-60-last.csv', 'ancient-tie-fighter-4-60-last.csv', 'clean-pyramid-7-60-last.csv', 'sweet-energy-9-60-last.csv']
['vibrant-rain-15-60-last.csv', 'trim-dawn-16-60-last.csv', 'vague-breeze-17-60-last.csv', 'cerulean-dawn-19-60-last.csv', 'helpful-sky-20-60-last.csv']
['benchmark-60-last.csv']


# Extract data for train set

In [6]:
multiplicative_path =  '../data/benchmark/train/multiplicative-0.99'
additive_path =  '../data/benchmark/train/additive-0.99'
rule_based_path =  '../data/benchmark/train/rule-based'

train_multi_profits_per_episode, train_multi_twb_df, train_multi_twb_df_ci, train_multi_dfs = compute_profits_twb(multiplicative_path)
train_multi_returns = compute_stats(train_multi_dfs)[-1]

train_additive_profits_per_episode, train_addtive_twb_df, train_addtive_twb_df_ci, train_additive_multi_dfs = compute_profits_twb(additive_path)
train_additive_returns = compute_stats(train_additive_multi_dfs)[-1]


train_rule_based_profits_per_episode, train_rule_based_twb_df, train_addtive_twb_df_ci, rule_based_multi_dfs = compute_profits_twb(rule_based_path)
# train_rule_based_returns = compute_stats(rule_based_multi_dfs)[-1]

['elegant-carrier-2-60-last.csv', 'rogue-podracer-3-60-last.csv', 'ancient-tie-fighter-4-60-last.csv', 'clean-pyramid-7-60-last.csv', 'sweet-energy-9-60-last.csv']
['vibrant-rain-15-60-last.csv', 'trim-dawn-16-60-last.csv', 'vague-breeze-17-60-last.csv', 'cerulean-dawn-19-60-last.csv', 'helpful-sky-20-60-last.csv']
['benchmark-60-last.csv']


In [7]:
np.mean(train_multi_returns), np.mean(test_multi_returns)

(1665.653486328125, 1635.5689986165364)

In [24]:
np.mean(train_multi_returns, axis=1), np.mean(test_multi_returns, axis=	1)


(array([1674.56743164, 1706.13344116, 1648.39363403, 1620.81319173,
        1678.35973307]),
 array([1640.0592041 , 1681.73196208, 1621.23024292, 1593.95943604,
        1640.86414795]))

In [8]:
train_multi_twb_df.mean(), test_multi_twb_df.mean()

(CO2 violation            99.518866
 Temperature violation    98.605440
 Humidity violation       69.202199
 dtype: float64,
 CO2 violation            99.694329
 Temperature violation    98.688889
 Humidity violation       68.413773
 dtype: float64)

In [26]:
train_multi_twb_df

,CO2 violation,Temperature violation,Humidity violation
elegant-carrier-2-60-last.csv,99.969329,98.523727,71.965278
rogue-podracer-3-60-last.csv,98.868634,99.086806,70.684606
ancient-tie-fighter-4-60-last.csv,99.915509,98.696759,70.225116
clean-pyramid-7-60-last.csv,99.977431,97.715856,62.655671
sweet-energy-9-60-last.csv,98.863426,99.004051,70.480324


In [9]:
np.mean(train_additive_returns), np.mean(test_additive_returns)

(1676.5717496744792, 1645.406474609375)

In [21]:
np.mean(train_additive_returns, axis=1), np.mean(test_additive_returns, axis=1)


(array([1660.23045044, 1696.46061808, 1679.71027425, 1686.80625   ,
        1659.6511556 ]),
 array([1621.89590047, 1662.97120565, 1652.58707072, 1660.38706258,
        1629.19113363]))

In [10]:
train_addtive_twb_df.mean(), test_addtive_twb_df.mean()

(CO2 violation            99.656250
 Temperature violation    99.623264
 Humidity violation       87.717477
 dtype: float64,
 CO2 violation            99.706944
 Temperature violation    99.675926
 Humidity violation       88.240856
 dtype: float64)

In [22]:
train_addtive_twb_df, test_addtive_twb_df


(                              CO2 violation  Temperature violation  \
 vibrant-rain-15-60-last.csv      100.000000              99.699653   
 trim-dawn-16-60-last.csv          99.525463              99.489583   
 vague-breeze-17-60-last.csv       99.692130              99.652199   
 cerulean-dawn-19-60-last.csv      99.373843              99.564236   
 helpful-sky-20-60-last.csv        99.689815              99.710648   
 
                               Humidity violation  
 vibrant-rain-15-60-last.csv            89.552662  
 trim-dawn-16-60-last.csv               89.576389  
 vague-breeze-17-60-last.csv            88.247685  
 cerulean-dawn-19-60-last.csv           84.914352  
 helpful-sky-20-60-last.csv             86.296296  ,
                               CO2 violation  Temperature violation  \
 vibrant-rain-15-60-last.csv      100.000000              99.865162   
 trim-dawn-16-60-last.csv          99.737847              99.454282   
 vague-breeze-17-60-last.csv       99.733796  

In [15]:
def compute_mean_profits(profits_per_episode):
    return np.array([df.mean()['Profits'] for df in profits_per_episode])

train_multi_profits = compute_mean_profits(train_multi_profits_per_episode)
test_multi_profits = compute_mean_profits(test_multi_profits_per_episode)

print(train_multi_profits.mean(), test_multi_profits.mean())
print(train_multi_profits, test_multi_profits)

3.8818851200703164 3.7879600670529237
[3.89222694 3.98844503 3.80042264 3.84705111 3.88127987] [3.80814382 3.9075376  3.70826994 3.74682173 3.76902725]


In [23]:
train_additive_profits = compute_mean_profits(train_additive_profits_per_episode)
test_additive_profits = compute_mean_profits(test_additive_profits_per_episode)

print(train_additive_profits, test_additive_profits)
print(train_additive_profits.mean(), test_additive_profits.mean())

[3.70083208 3.81797977 3.76203187 3.8390939  3.7147926 ] [3.57849703 3.7053569  3.65816119 3.74262091 3.61560591]
3.766946045159885 3.660048387292508


In [13]:
print(train_rule_based_twb_df)
test_rule_based_twb_df

                       CO2 violation  Temperature violation  \
benchmark-60-last.csv       99.89294              99.904514   

                       Humidity violation  
benchmark-60-last.csv           83.077546  


,CO2 violation,Temperature violation,Humidity violation
benchmark-60-last.csv,99.938657,99.956308,84.099826


In [14]:
train_rule_based_profits = compute_mean_profits(train_rule_based_profits_per_episode)
test_rule_based_profits = compute_mean_profits(test_rule_based_profits_per_episode)
print(train_rule_based_profits.mean(), test_rule_based_profits.mean())

3.9712523569443565 3.9023978349604183
